In [ ]:
from datasetsforecast.long_horizon import LongHorizon
import pandas as pd

Y_df, X_df, S_df = LongHorizon.load(directory='data', group='ETTh1')

Y_df["ds"] = pd.to_datetime(Y_df["ds"])
X_df["ds"] = pd.to_datetime(X_df["ds"])

df = pd.concat([Y_df[["y"]], X_df], axis=1)
df

In [ ]:
import logging

from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS, RNN, TFT
from neuralforecast.auto import AutoLSTM, AutoNHITS, AutoRNN, AutoTFT

logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)

In [ ]:
horizon = 12

exog_list = ['ex_1', 'ex_2', 'ex_3', 'ex_4']

# Try different hyperparmeters to improve accuracy.
models = [
      AutoLSTM(
            h=horizon,
            backend='optuna',
            config=AutoLSTM.get_default_config(h=horizon, backend='optuna')
      ),
      AutoNHITS(
            h=horizon,
            backend='optuna',
            config=AutoNHITS.get_default_config(h=horizon, backend='optuna')
      ),
      AutoTFT(
            h=horizon,
            backend='optuna',
            config=AutoTFT.get_default_config(h=horizon, backend='optuna')
      )
]
nf = NeuralForecast(models=models, freq='h')
nf.fit(df=df)

In [ ]:
results = nf.models[0].results.trials_dataframe()
results

In [ ]:
nf.save(
    path="./saved_models/initial_run",
    model_index=None, 
    overwrite=True,
    save_dataset=True
)